In [1]:
import mne
import re
import os
import numpy as np
import source_localization as srcl
import MEG_analysis as meg

In [2]:
home = '../../../../MEG_raw01/'
home_folders = os.listdir(home)
home_folders = [os.path.join(home, entry) for entry in home_folders if re.findall(r"^\w+", entry)]
folder_dict = {}
for folder in home_folders:
    entries = os.listdir(folder)
    folder_dict[folder] = [os.path.join(folder, entry) for entry in entries if re.findall(r"^\w+SD_\w*raw.fif", entry)]

subj = 'NNo'
meg_dir = '../../../../MEG_raw01/170808_noguchi_SD'
load = False
source_localization_dir = "/usr/local/freesurfer/subjects"

In [3]:
evoked, epochs, info = meg.process_data(folder_dict[meg_dir], 0)
#evoked.save('../Data/Evoked/%s-ave.fif' % subj)
#epochs.save('../Data/Epochs/%s-epo.fif' % subj)
#src = srcl.create_source_space(subj, source_localization_dir, save=True)
#bem = srcl.make_bem(subj, source_localization_dir, save=True)
src = srcl.get_source_space("../Data/SourceSpaces/%s-oct6-src.fif" % subj)

Executing ICA
Fitting ICA to data using 204 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Selection by number: 20 components
Fitting ICA took 75.1s.
Transforming to ICA space (20 components)
Zeroing out 2 ICA components
Epoching Data
847 events found
Event IDs: [16384 32768]
801 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
Created an SSP operator (subspace dimension = 5)
10 projection items activated
Loading data for 801 events and 1501 original time points ...


/media/whitney/New Volume/Yuki/MNE_analysis/honors-thesis/honors-thesis/code/MEG_analysis.py:51: RuntimeWarning: Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug
  events = mne.find_events(data)
/media/whitney/New Volume/Yuki/MNE_analysis/honors-thesis/honors-thesis/code/MEG_analysis.py:52: RuntimeWarning: Projection vector "grad_68_noIAS_20140702-2.fif : PCA-v1" has magnitude 0.91 (should be unity), applying projector with 204/240 of the original channels available may be dangerous, consider recomputing and adding projection vectors for channels that are eventually used. If this is intentional, consider using info.normalize_proj()
  epochs = mne.Epochs(data, events, event_id=16384, tmin=-0.5, tmax = 1, reject = reject)
/media/whitney/New Volume/Yuki/MNE_analysis/honors-thesis/honors-thesis/code/MEG_analysis.py:52: RuntimeWarning: Projection vect

0 bad epochs dropped
Plotting Evoked
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read


In [ ]:
mne.viz.plot_alignment(info, trans=None, subject=subj, src=src,
                       subjects_dir=source_localization_dir, dig=True, 
                       surfaces=['white'], coord_frame='meg')

Getting helmet for system 306m


In [ ]:
cov = mne.compute_covariance(epochs, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=True)
fwd = srcl.make_forward_sol(evoked, src, bem)
inv_op = srcl.make_inverse_operator(evoked, fwd, cov)
stc, residual = srcl.apply_inverse(evoked, inv_op)
#residual.plot_topo(title='Residual Plot', show=False).savefig('../Figures/%s_residual_erf.pdf' % subj)
stc_fsaverage = srcl.morph_to_fsaverage(stc, subj)